In [114]:
import random

class Sentiment:
    POSITIVE = 'POSITIVE'
    NEUTRAL = 'NEUTRAL'
    NEGATIVE = 'NEGATIVE'

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score >=4:
            return Sentiment.POSITIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.NEGATIVE

class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment==Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment==Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)
        

In [115]:
import json

file_name = './data/sentiment/Books_small_10000.json'

reviews = []
with open(file_name) as file:
    for line in file:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

In [116]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

In [117]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()


test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

In [118]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

train_x_vectors = vectorizer.fit_transform(train_x)

# Don't use fit cause we just did this, just transform our test_x
test_x_vectors = vectorizer.transform(test_x)


In [119]:
print(training[0].sentiment)

POSITIVE


#### Linear SVM

In [120]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

clf_svm.predict(test_x_vectors[0])


array(['NEGATIVE'], dtype='<U8')

#### Decision Tree


In [121]:
from sklearn.tree import DecisionTreeClassifier

clf_tree = DecisionTreeClassifier()

clf_tree.fit(train_x_vectors, train_y)

clf_tree.predict(test_x_vectors[0])



array(['NEGATIVE'], dtype='<U8')

In [122]:
from sklearn.metrics import accuracy_score

# There are two methods of checking accuracy

# prediction = clf_tree.predict(test_x_vectors)
# accuracy_score(test_y, prediction)

clf_tree.score(test_x_vectors, test_y)

0.6322115384615384

In [123]:
prediction = clf_svm.predict(test_x_vectors)
accuracy_score(test_y, prediction)

0.7980769230769231

In [124]:
from sklearn.metrics import f1_score

f1_score(test_y,
         clf_svm.predict(test_x_vectors),
         average=None,
         labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])

array([0.8028169 , 0.79310345])

In [125]:
f1_score(test_y,
         clf_tree.predict(test_x_vectors),
         average=None,
         labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])

array([0.63829787, 0.62591687])

In [126]:
train_y.count(Sentiment.NEGATIVE)

436

#### Test of our model

In [127]:
test_set = ['This movie was awful', 'I am in love', 'This programmer is so stupid and useful']
new_test = vectorizer.transform(test_set)
clf_svm.predict(new_test)

array(['NEGATIVE', 'POSITIVE', 'NEGATIVE'], dtype='<U8')